In [51]:
# importing the dependencies
from itertools import product as itp
from pythtb import *
from triqs.lattice.tight_binding import TBLattice
import sympy as sp
import warnings
import numpy as np
import matplotlib.pyplot as plt

In [52]:
def TB_to_sympy_2D(TBL, analytical = True, precision = 6):
    r"""
    returns the analytical form of the momentum space hamiltonian of the tight-binding model 
    from a tight-binding lattice object by utilizing Fourier series
    
    Parameters
    ----------
    TBL: triqs TBLattice object
        triqs tight binding object
    analytical: boolean, default = True
        a boolean which will cause the function will return an analytical Hamiltonian, when true, and 
        an numerical Hamiltonian otherwise
    precision: integer, default = 6
        specifies the precision of the floating point amplitudes. The default precision is 6 but the user
        can decrease this parameter for better recognition of similar hopping amplitudes, particularly
        for symmetrical hoppings across the crystal lattice
    
    Returns
    -------
    Hk: NumPy array
        the Hamiltonian of the tight-binding model in momentum space. It can be output in either numerical
        form (Hk_numerical) or reduced analytical form (Hk) based on the user's choice. The default output
        is the reduced analytical form. The numerical form depends solely on the k-space vector components
        while the analytical form takes into account both the k-space vector components and the lattice
        vectors

    """

    import sympy as sp

    # imaginary number
    I = sp.I

    # matrix from axis directions in momentum space
    kx, ky = sp.symbols("kx ky", real = True)
    k_space_matrix = sp.Matrix([kx, ky])

    # symbolic dot product representation between lattice unit vectors
    # and momentum space matrix
    a1k, a2k = sp.symbols("a1k a2k", real = True)
    lattice = sp.Matrix([a1k, a2k])

    # number of orbitals involved in the unit cell
    num_orb = TBL.n_orbitals

    # dictionary containing details about hopping of electrons such as
    # orbital and hopping amplitude info
    TBL_hops = TBL.hoppings

    # maximum hopping distances of electrons in each direction
    max_x, max_y = list(np.max(np.array(list(TBL_hops.keys())), axis = 0))

    # number of cells involved in the hopping of electrons in each direction
    num_cells_x, num_cells_y = [2 * max_coord + 1 for max_coord in [max_x, max_y]]
    
    # real-space Hamiltonian
    Hrij = np.zeros((num_cells_x, num_cells_y, num_orb, num_orb), dtype = sp.exp)

    # looping through hopping parameters of electrons involved in inter-orbital hoppings
    for key, hopping in TBL_hops.items():
        rx, ry= key
        hopping = np.around(hopping, precision)
        Hrij[rx + max_x, ry + max_y] = hopping

    # basis of the exponential term in calculation of Hk
    Hexp = np.empty_like(Hrij, dtype = sp.exp)

    # perform Fourier transform
    for xi, yi in itp(range(num_cells_x), range(num_cells_y)):
        coefficients = np.array([xi - max_x, yi - max_y])
        r = lattice.dot(coefficients)
        eikr = sp.exp(-I * r)
        Hexp[xi, yi, :, :] = eikr

    # summation over all real space axes
    Hk = np.sum(Hrij * Hexp, axis = (0, 1))

    # rewriting exponential terms in Hamiltonian expression in terms of cosine
    for i, j in itp(range(num_orb), repeat = 2):
        Hk[i, j] = Hk[i, j].rewrite(sp.cos)

    # dealing with the numerical Hamiltonian

    # convert to SymPy matrix to use substitutions method available in SymPy
    Hk_numerical = sp.Matrix(Hk)

    # matrix containing displacement vectors
    TBL_units = TBL.units

    # obtaining individual displacement vectors
    a1 = np.around(TBL_units[0], precision)
    a2 = np.around(TBL_units[1], precision)

    # numerical dot products between unit vectors
    # and momentum space matrix
    a1k_numerical = a1.dot(k_space_matrix)[0]
    a2k_numerical = a2.dot(k_space_matrix)[0]
    
    # performing numerical dot product substitutions
    Hk_numerical = Hk_numerical.subs(a1k, a1k_numerical)
    Hk_numerical = Hk_numerical.subs(a2k, a2k_numerical)

    # converting numerical Hamiltonian to NumPy array
    Hk_numerical = np.array(Hk_numerical)

    def _has_complex_exponential_sympy(matrix):
        """
        Checks if a NumPy array containing SymPy elements has a complex exponential element.

        Args:
            matrix (NumPy array): The input NumPy array containing SymPy elements
        
        Returns:
            bool: True if the matrix array contains a complex exponential element, False otherwise.

        """

        for sublist in matrix:
            for element in sublist:
                if element.is_complex and element.has(sp.exp):
                    return True
        return False
    
    def _is_hermitian_sympy(matrix):
        """
        Checks if a NumPy array containing SymPy elements is hermitian

        Args:
            matrix (NumPy array): The input NumPy array containing SymPy elements
        
        Returns:
            bool: True if the matrix is a hermitian, False otherwise

        """
        
        n = matrix.shape[0]
        for i in range(n):
            for j in range(n):
                if matrix[i, j] != matrix[j, i].conjugate():
                    return False
        return True
    
    # warning indicating when Hamiltonian is not hermitian
    if _is_hermitian_sympy(Hk) == False or _is_hermitian_sympy(Hk_numerical) == False:
        return warnings.warn("The resulting Hamiltonian is not hermitian.")

    # warning indicating when Hamiltonian contains complex exponential element
    if _has_complex_exponential_sympy(Hk_numerical) or _has_complex_exponential_sympy(Hk):
        return warnings.warn("""Your expression has a complex exponential. 
                                Choosing a different unit cell could make 
                                your Hamiltonian expression real.""")
    
    # returning analytical or numerical form of Hamiltonian
    if analytical:
        return Hk
    return Hk_numerical

In [60]:
units = [(2, 1), (1, 2)]
norb = 2
hoppings = {(0, 1): np.eye(norb),
            (0, -1): np.eye(norb),
            (1, 0):  np.ones((norb, norb)),
            (-1, 0): np.ones((norb, norb))
            }
orbital_positions = [(0, 0)] * norb
orbital_names = [""] * norb


simpleTBLattice = TBLattice(units = units, hoppings = hoppings, orbital_positions = orbital_positions, orbital_names = orbital_names)

# calling the function on TB_to_sympy
simpleModelHamAnalytical = TB_to_sympy_2D(simpleTBLattice, analytical = True, precision = 3)
simpleModelHamNumerical = TB_to_sympy_2D(simpleTBLattice, analytical = False, precision = 3)
simpleModelHamNumericalMorePrecise = TB_to_sympy_2D(simpleTBLattice, analytical = False, precision = 5)
print(simpleModelHamNumericalMorePrecise)
print(simpleModelHamAnalytical)
simpleModelHamAnalyticalAsList = simpleModelHamAnalytical.tolist()
print(simpleModelHamAnalyticalAsList)

print(list(simpleModelHamAnalytical))
print(type(simpleModelHamAnalytical))
print()
print(simpleModelHamNumerical)
simpleModelHamNumericalAsList = simpleModelHamNumerical.tolist()
print(simpleModelHamNumericalAsList)

[[2.0*cos(1.0*kx + 2.0*ky) + 2.0*cos(2.0*kx + 1.0*ky)
  2.0*cos(2.0*kx + 1.0*ky)]
 [2.0*cos(2.0*kx + 1.0*ky)
  2.0*cos(1.0*kx + 2.0*ky) + 2.0*cos(2.0*kx + 1.0*ky)]]
